In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
       # print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from fastai.vision.all import *

# learning by doing: referenced from https://github.com/fastai/fastbook/blob/master/06_multicat.ipynb



In [ ]:
# get and define the paths
cassavaPath = '../input/cassava-leaf-disease-classification/'
cassavaOutputPath = './'

cassavaModelPath = '../input/cassavleafdiseaseclassificationpretrained/'

# load in the data
df = pd.read_csv(cassavaPath+'train.csv')
df['label'] = df['label'].astype(str)

# check if it looks legit
df.head()

In [ ]:
# quick check how the data is distributed
df['label'].value_counts()

In [ ]:
#define the path to the input files and to the respective labels
def get_x(r): return cassavaPath+'train_images/'+r['image_id']
def get_y(r): return r['label']

dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(), #use the default random splitter to split between train and test set
                   get_x=get_x,
                   get_y=get_y,
                   item_tfms=Resize(600),
                   batch_tfms=aug_transforms())

# just to check how the dataset looks like
dsets = dblock.datasets(df)
dsets.train[0]


In [ ]:
# use dataloaders to prepare the pytorch batches
dls = dblock.dataloaders(df)

In [ ]:
# quick check how the batches look like and if the labels match with the input data
dls.show_batch(nrows=3, ncols=3)

In [ ]:
# kaggle notebook does not have the ressources. I trained it with my Azure DSVM. Code is here for reference

#learn = cnn_learner(dls, resnet34, metrics=error_rate)
#learn.fine_tune(7, base_lr=3e-3) #epoch number and learn rate are a wild guess. I'm still a beginner # ;)

In [ ]:
# kaggle notebook does not have the ressources. I trained it with my Azure DSVM. Code is here for reference

#learn.export('CassavaDiseaseModelResnet34.pkl') 

In [ ]:
# kaggle notebook does not have the ressources. I trained it with my Azure DSVM. Code is here for reference. 
# This is to check how good/bad the model performs on the test set. To be honest a lot of confusions ... does not perform so well.
# possibly this could be improved by having a form of segmentation pre-processing ... ? So that the model really focuses on the plants and not on soil and sky.

#interp = ClassificationInterpretation.from_learner(learn)
#interp.plot_confusion_matrix(figsize=(5,5))

In [ ]:
# kaggle notebook does not have the ressources. I trained it with my Azure DSVM. Code is here for reference. 
# Just to check how some examples look like

#learn.show_results()

In [ ]:
# load my pre-trained model. commented code above in case someone wants to reproduce it

learn = load_learner(cassavaModelPath+'CassavaDiseaseModelResnet34.pkl')
# move the model in the output folder (the first submission failed. Not sure if it is related to the access to the pretrained model)
learn.export(cassavaOutputPath+'CassavaDiseaseModelResnet34.pkl')
learn = load_learner(cassavaOutputPath+'CassavaDiseaseModelResnet34.pkl')

In [ ]:
print('Computing predictions...')
test_files = get_image_files(cassavaPath+'test_images')


predictions_ResultArray = [None for tempX in range(len(test_files))]
predictions_InputArray = [None for tempX in range(len(test_files))]

for test_idx in range(0,len(test_files)):
    predictions = learn.predict(test_files[test_idx])
    head, tail = os.path.split(test_files[test_idx])
    predictions_ResultArray[test_idx] = int(predictions[0])
    predictions_InputArray[test_idx] = tail
     
np.savetxt(cassavaOutputPath+'submission.csv', np.rec.fromarrays([predictions_InputArray, predictions_ResultArray]), fmt=['%s', '%d'], delimiter=',', header='image_id,label', comments='')
    

    
!head submission.csv

In [ ]:
# double check if the newly generated output file matches the template result file

df_sample_submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
df_exact_submission = pd.read_csv(cassavaOutputPath+'submission.csv')

#print(df_sample_submission)
#print(df_exact_submission)

print(df_sample_submission.compare(df_exact_submission))

print(df_sample_submission.equals(df_exact_submission))
